### DATA102 Final Project
Submitted by **ALDECOA**, Renzel; **DUCUT**, Ezekiel; **HIDALGO**, Francisco; and **RECOMONO**, Francis <br>
Group **8** - S13

---
#### Statement of the Problem

Lorem ipsum

---
#### Importing the necessary libraries

In [6]:
import pandas as pd
import numpy as np

---
#### Loading the dataset

The <a href="https://www.kaggle.com/datasets/justinas/nba-players-data">NBA Players</a> dataset contains over two decades of data on each player who has been part of an NBA teams' roster. It captures demographic variables such as age, height, weight and place of birth, biographical details like the team played for, draft year and round. In addition, it has basic box score statistics such as games played, average number of points, rebounds, assists, etc.

In [9]:
df = pd.read_csv("NBA-Dataset/all_seasons.csv", index_col=0)
df.head()

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,Randy Livingston,HOU,22.0,193.04,94.800728,Louisiana State,USA,1996,2,42,...,3.9,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996-97
1,Gaylon Nickerson,WAS,28.0,190.50,86.182480,Northwestern Oklahoma,USA,1994,2,34,...,3.8,1.3,0.3,8.9,0.030,0.111,0.174,0.497,0.043,1996-97
2,George Lynch,VAN,26.0,203.20,103.418976,North Carolina,USA,1993,1,12,...,8.3,6.4,1.9,-8.2,0.106,0.185,0.175,0.512,0.125,1996-97
3,George McCloud,LAL,30.0,203.20,102.058200,Florida State,USA,1989,1,7,...,10.2,2.8,1.7,-2.7,0.027,0.111,0.206,0.527,0.125,1996-97
4,George Zidek,DEN,23.0,213.36,119.748288,UCLA,USA,1995,1,22,...,2.8,1.7,0.3,-14.1,0.102,0.169,0.195,0.500,0.064,1996-97


##### Dataset infomation via `pandas`

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12844 entries, 0 to 12843
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   player_name        12844 non-null  object 
 1   team_abbreviation  12844 non-null  object 
 2   age                12844 non-null  float64
 3   player_height      12844 non-null  float64
 4   player_weight      12844 non-null  float64
 5   college            10990 non-null  object 
 6   country            12844 non-null  object 
 7   draft_year         12844 non-null  object 
 8   draft_round        12844 non-null  object 
 9   draft_number       12844 non-null  object 
 10  gp                 12844 non-null  int64  
 11  pts                12844 non-null  float64
 12  reb                12844 non-null  float64
 13  ast                12844 non-null  float64
 14  net_rating         12844 non-null  float64
 15  oreb_pct           12844 non-null  float64
 16  dreb_pct           12844 no

##### Feature Descriptions

- `player_name` - Name of the player
- `team_abbreviation` - Abbreviated name of the tead the player played for (at the end of the season)
- `age` - Age of the player
- `player_height` - Height of the player (in centimeters)
- `player_weight` - Weight of the player (in kilograms)
- `college` - Name of the college the player attended
- `country` - Name of the country the player was born in (not necessarily the nationality)
- `draft_year` - Year the player was drafted
- `draft_round` - Draft round the player was picked
- `draft_number` - Number at which the player was picked in his draft round
- `gp` - Games played (throughout current season)
- `pts` - Average number of points scored per game
- `reb` - Average number of rebounds grabbed per game
- `ast` - Average number of assists distributed per game
- `net_rating` - Team's point differential per 100 possessions while the player is on court
- `oreb_pct` - Percentage of available offensive rebounds the player grabbed while he was on court
- `dreb_pct` - Percentage of available defensive rebounds the player grabbed while he was on court
- `usg_pct` - Percentage of team plays used by the player while he was on court ((FGA + Possession Ending FTA + TO) / POSS)
- `ts_pct` - Measure of the player's shooting efficiency that takes into account free throws, 2 and 3 point shots (PTS / (2 * (FGA + 0.44 * FTA)))
- `ast_pct` - Percentage of teammate field goals the player assisted while he was on court
- `season` - NBA season

*Descriptions taken from Kaggle dataset details*

---
#### Exploratory Data Analysis

In [15]:
df.head(3)

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,Randy Livingston,HOU,22.0,193.04,94.800728,Louisiana State,USA,1996,2,42,...,3.9,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996-97
1,Gaylon Nickerson,WAS,28.0,190.50,86.182480,Northwestern Oklahoma,USA,1994,2,34,...,3.8,1.3,0.3,8.9,0.030,0.111,0.174,0.497,0.043,1996-97
2,George Lynch,VAN,26.0,203.20,103.418976,North Carolina,USA,1993,1,12,...,8.3,6.4,1.9,-8.2,0.106,0.185,0.175,0.512,0.125,1996-97


**Unique values**

In [17]:
player_num = len(df["player_name"].unique())
college_num = len(df["college"].unique())
season_num = len(df["season"].unique())

print(f"Number of unique players (across all seasons): {player_num}")
print(f"Number of unique colleges: {college_num}")

Number of unique players (across all seasons): 2551
Number of unique colleges: 357


**Draft rounds and numbers** <br>
These values indicate at what order did the player get drafted into a team for the current season. Being picked first, either in the first round (`draft_round` = 1) or the first in a round (`draft_number` = 1), can be a sign of acknowldegement by the NBA teams for a player's talent on the court (<a href="https://www.bbc.co.uk/newsround/65997816">BBC, 2023</a>). 

In [19]:
df["draft_round"].unique()

array(['2', '1', 'Undrafted', '3', '4', '8', '6', '7', '0'], dtype=object)

In [20]:
df.loc[df["draft_round"] == '0']

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
10651,Jeremy Pargo,GSW,34.0,187.96,99.336648,Gonzaga,USA,2009,0,0,...,8.3,1.0,2.7,2.4,0.024,0.048,0.268,0.557,0.308,2019-20
10791,Josh Magette,ORL,30.0,185.42,72.574720,Alabama Huntsville,USA,2012,0,0,...,1.5,0.8,0.6,2.4,0.022,0.128,0.226,0.378,0.238,2019-20
10966,T.J. McConnell,IND,28.0,185.42,86.182480,Arizona,USA,2015,0,Undrafted,...,6.5,2.7,5.0,1.1,0.028,0.107,0.167,0.540,0.354,2019-20
11498,T.J. McConnell,IND,29.0,185.42,86.182480,Arizona,USA,2015,0,Undrafted,...,8.6,3.7,6.6,2.5,0.030,0.106,0.149,0.583,0.337,2020-21
12247,T.J. McConnell,IND,30.0,185.42,86.182480,Arizona,USA,2015,0,Undrafted,...,8.5,3.3,4.9,-5.5,0.029,0.110,0.157,0.525,0.287,2021-22
12266,Scotty Hopson,OKC,32.0,200.66,92.532768,Tennessee,USA,2011,0,0,...,4.0,1.0,1.0,-18.4,0.000,0.043,0.143,0.500,0.091,2021-22


The number `0` and the word `Undrafted` under the columns `draft_round` and `draft_number` mean the same thing, so we map the rows containing `Undrafted` with `0` to remain consistent with the data type and meaning of the values.

In [22]:
df["draft_round"] = df["draft_round"].replace({
    "Undrafted": 0
})

df["draft_number"] = df["draft_number"].replace({
    "Undrafted": 0
})

# Converting the values in draft_round and draft_number into integers
df["draft_round"] = pd.to_numeric(df["draft_round"])
df["draft_number"] = pd.to_numeric(df["draft_number"])

In [23]:
print(df["draft_round"].unique())
print(df["draft_number"].unique())

[2 1 0 3 4 8 6 7]
[ 42  34  12   7  22  47  30   4   1   3  11  33  26  35  17  25  28  49
  23   2  15  27   0  37  38  13  18   9  21  43  14  10   6  57  52  32
  39  46  29  16  45  20  48   5  79  41  51  36  75  56   8  24 165  40
  19  58  44 139  54 127 124  50 160  63  31  69  53  55 120  87  66  82
  60  59  78]


---
#### Data Preprocessing

---
#### Model Training